In [2]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
from transformers import ResNetForImageClassification
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn import Sequential as SequentialTorch, Flatten as FlattenTorch
from sklearn.svm import NuSVC, SVC
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import ast

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

In [3]:
model_checkpoint = 'microsoft/resnet-50'
model = ResNetForImageClassification.from_pretrained(model_checkpoint)

model.classifier = SequentialTorch(
    FlattenTorch(start_dim=1, end_dim=-1)
)

In [9]:
orb = cv.ORB_create(64)

In [10]:
folder_path = "./test_data"

In [11]:
features = []
labels = []
handcrafted_features = []
deep_features = []

In [12]:
def extract_label_from_image_name(image_name):
    # Assuming the label is encoded in the last 9 characters
    label = image_name[-14:-5]
    return label

In [13]:
print(len(os.listdir(folder_path)))

50000


In [14]:
for image_name in os.listdir(folder_path):
    # Assuming image names have a code that corresponds to the label
    label = extract_label_from_image_name(image_name)

    # Load and preprocess the image
    image_path = os.path.join(folder_path, image_name)
    img = cv.imread(image_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # ORB feature extraction
    kp = orb.detect(img, None)
    if kp:
        kp, des = orb.compute(img, kp)
        handcrafted_features.append(des.flatten())

        # Check if descriptors are not None and not empty
        if des is not None and des.size > 0:
            # ResNet-50 feature extraction
            image = Image.open(image_path).resize((224, 224))
            if image.mode != 'RGB':
                image = image.convert('RGB')
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = transform(image)
            inputs = image.unsqueeze(0)
            outputs = model(inputs)
            logits = outputs.logits
            resnet_features = logits.detach().numpy().flatten()
            deep_features.append(resnet_features)

            # Check if resnet_features is not empty
            if resnet_features.size > 0:
                # Combine features
                # combined_features = np.stack((resnet_features, des.flatten()), axis=1)

                # Append to lists
                # features.append(combined_features)
                labels.append(label)

In [23]:
print(np.array(handcrafted_features).shape)


In [24]:
for i in range(len(handcrafted_features)):
    if type(handcrafted_features[i]) != list:
        handcrafted_features[i] = handcrafted_features[i].tolist()
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))
        handcrafted_features[i] = handcrafted_features[i] + [0]*(2048 - len(handcrafted_features[i]))
        
print('loop done')
        
for i in range(len(handcrafted_features)):
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))

1248
352
512
1984
1728
1632
1536
1984
1984
1952
1984
1088
2016
1984
1760
2016
640
1152
1824
2016
1824
1248
960
1888
1984
1920
1664
1952
1984
1632
1824
1888
1984
1664
1984
1824
2016
1728
1984
2016
1920
992
1856
1920
1856
1984
608
1440
1984
1952
2016
2016
1984
1824
2016
1984
768
1696
1920
1216
1984
1824
2016
2016
1280
2016
1984
1600
2016
960
832
1536
1504
1472
256
1984
1408
1568
1824
1824
1984
832
1632
1536
1984
1600
1632
1984
1152
1824
832
1984
2016
1632
1792
1984
1792
1984
1408
1984
2016
1824
480
1664
288
1824
992
1632
1824
1664
1376
1440
864
1984
1856
1856
1536
1984
224
1952
1984
2016
1888
1984
1888
832
1088
1984
1888
1760
1056
2016
1472
2016
1952
1952
1152
1824
1344
1216
2016
672
1824
1920
1248
1984
1568
800
1792
1632
1824
1952
1248
864
2016
1152
2016
1408
2016
2016
608
1952
1920
1728
1856
1984
1824
1824
1920
1824
1632
1856
1984
1952
608
1984
1888
1728
1984
1792
768
1952
1440
1984
2016
1984
1312
1568
1984
1856
1760
2016
1664
1920
1888
672
1952
1632
1984
1984
1984
1504
2016
1824
800
5

In [25]:
print(np.array(handcrafted_features).shape)
print(np.array(deep_features).shape)

(49982, 2048)
(49982, 2048)


In [26]:
handcrafted_features = np.array(handcrafted_features)
deep_features = np.array(deep_features)

In [27]:
combined_features = np.stack((deep_features, handcrafted_features), axis=1)

In [28]:
combined_features.shape

(49982, 2, 2048)

In [31]:
combined_features = combined_features.reshape((49982, 2, 2048, 1))
print(combined_features.shape)

(49982, 2, 2048, 1)


In [34]:
# class CNN(nn.Module):
#     def __init__(self, in_channels: int = 1, num_classes: int = 1000):
#         super(CNN, self).__init__()
        
#         self.network = nn.Sequential(
            
#             nn.Conv2d(in_channels, 32, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 2)),
            
#             nn.Conv2d(64, 128, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.Conv2d(128, 128, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 2)),
            
#             nn.Conv2d(128, 128, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.Conv2d(128, 256, kernel_size=(1, 3), bias=False),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1, 2)),
            
#             nn.Flatten(),
            
#             nn.Linear(504, 2048),
#             nn.ReLU(),
#             nn.Linear(2048, 2048),
#             nn.ReLU(),
            
#             nn.Linear(2048, 1000),
#             nn.Softmax()
#         )
        
#     def forward(self, x):
#         return self.network(x)


# class CNN(nn.Module):
#   def __init__(self, in_channels: int = 1, num_classes: int = 1000):
#     super(CNN, self).__init__()
    
#     self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(1,3), stride=(1,1))
#     self.pool = nn.MaxPool2d(kernel_size=(1,2), stride=(1,2))
#     self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(1,3), stride=(1,1))
#     self.fc1 = nn.Linear(in_features=16320, out_features=num_classes)

#   def forward(self, x):
#     x = F.relu(self.conv1(x))
#     x = self.pool(x)
#     x = F.relu(self.conv2(x))
#     x = self.pool(x)
    
#     x = x.reshape(x.shape[0], -1)
    
#     x = self.fc1(x)
    
#     return x

model = Sequential([
    Conv2D(16, (1, 3), activation='relu', input_shape=(2, 2048, 1)),
    BatchNormalization(),
    Conv2D(32, (1, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(1, 2)),
    Dropout(0.25),
    
    Conv2D(32, (1, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (1, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(1, 2)),
    Dropout(0.25),
    
    Conv2D(64, (1, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(32, (1, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(1, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dropout(0.25),
    
    Dense(128, activation='relu'),
    Dropout(0.25),
    
    Dense(1000, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 2046, 16)       64        
                                                                 
 batch_normalization (Batch  (None, 2, 2046, 16)       64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2044, 32)       1568      
                                                                 
 batch_normalization_1 (Bat  (None, 2, 2044, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 1022, 32)       0         
 D)                                                              
                                                        

In [35]:
file_path = './LOC_synset_mapping.txt'

with open(file_path, 'r') as file:
    class_to_name_dict = {}
    for line in file:
        line = line.strip()
        if line:
            parts = line.split(' ', 1)
            class_label = parts[0]
            class_name = parts[1]
            
            class_to_name_dict[class_label] = class_name

In [36]:
file_path = './imagenet1000_clsidx_to_labels.txt'

with open(file_path, 'r') as file:
    content = file.read()
    extracted_dict = ast.literal_eval(content)

class_name_to_numeric_class_dict = {value: key for key, value in extracted_dict.items()}

In [37]:
print(class_name_to_numeric_class_dict)
print(class_to_name_dict)
print(labels)
print(len(labels))

{'tench, Tinca tinca': 0, 'goldfish, Carassius auratus': 1, 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias': 2, 'tiger shark, Galeocerdo cuvieri': 3, 'hammerhead, hammerhead shark': 4, 'electric ray, crampfish, numbfish, torpedo': 5, 'stingray': 6, 'cock': 7, 'hen': 8, 'ostrich, Struthio camelus': 9, 'brambling, Fringilla montifringilla': 10, 'goldfinch, Carduelis carduelis': 11, 'house finch, linnet, Carpodacus mexicanus': 12, 'junco, snowbird': 13, 'indigo bunting, indigo finch, indigo bird, Passerina cyanea': 14, 'robin, American robin, Turdus migratorius': 15, 'bulbul': 16, 'jay': 17, 'magpie': 18, 'chickadee': 19, 'water ouzel, dipper': 20, 'kite': 21, 'bald eagle, American eagle, Haliaeetus leucocephalus': 22, 'vulture': 23, 'great grey owl, great gray owl, Strix nebulosa': 24, 'European fire salamander, Salamandra salamandra': 25, 'common newt, Triturus vulgaris': 26, 'eft': 27, 'spotted salamander, Ambystoma maculatum': 28, 'axolotl, mud pu

In [38]:
mapped_labels = []

for i in range(len(labels)):
    class_name_for_label = class_to_name_dict[labels[i]]
    numeric_label = class_name_to_numeric_class_dict[class_name_for_label]
    mapped_labels.append(numeric_label)
    
print(mapped_labels)

[65, 970, 230, 809, 516, 57, 334, 415, 674, 332, 109, 286, 370, 757, 595, 147, 108, 23, 478, 517, 334, 173, 948, 727, 23, 846, 270, 167, 55, 858, 324, 573, 150, 981, 586, 887, 32, 398, 777, 74, 516, 756, 129, 198, 256, 725, 565, 167, 717, 394, 92, 29, 844, 591, 358, 468, 259, 994, 872, 588, 474, 183, 107, 46, 842, 390, 101, 887, 870, 841, 467, 149, 21, 476, 80, 424, 159, 275, 175, 461, 970, 160, 788, 58, 479, 498, 369, 28, 487, 50, 270, 383, 366, 780, 373, 705, 330, 142, 949, 349, 473, 159, 872, 878, 201, 906, 70, 486, 632, 608, 122, 720, 227, 686, 173, 959, 638, 646, 664, 645, 718, 483, 852, 392, 311, 457, 352, 22, 934, 283, 802, 553, 276, 236, 751, 343, 528, 328, 969, 558, 163, 328, 771, 726, 977, 875, 265, 686, 590, 975, 620, 637, 39, 115, 937, 272, 277, 763, 789, 646, 213, 493, 647, 504, 937, 687, 781, 666, 583, 158, 825, 212, 659, 257, 436, 196, 140, 248, 339, 230, 361, 544, 935, 638, 627, 289, 867, 272, 103, 584, 180, 703, 449, 771, 118, 396, 934, 16, 548, 993, 704, 457, 233, 401

In [39]:
mapped_labels = np.array(mapped_labels)

X_train, X_test, y_train, y_test = train_test_split(combined_features, mapped_labels, test_size=0.1, random_state=100)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(44983, 2, 2048, 1) (44983,) (4999, 2, 2048, 1) (4999,)


In [40]:
# # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

# in_channels = 1
# num_classes = 1000
# learning_rate = 0.001
# batch_size = 64
# num_epochs = 10

# X_train = torch.Tensor(X_train)
# y_train = torch.Tensor(y_train)
# X_test = torch.Tensor(X_test)
# y_test = torch.Tensor(y_test)

# train_loader_tensor = torch.utils.data.TensorDataset(X_train, y_train)
# test_loader_tensor = torch.utils.data.TensorDataset(X_test, y_test)

# train_loader = DataLoader(dataset=train_loader_tensor, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=test_loader_tensor, batch_size=batch_size, shuffle=True)

# model = CNN().to(device=device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(lr=learning_rate, params=model.parameters())

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [41]:
# for epoch in range(num_epochs):
#     print(f'Running epoch number {epoch}...')
#     for batch_idx, (data, targets) in enumerate(train_loader):
#         targets = targets.type(torch.LongTensor)
        
#         data = data.to(device=device)
#         targets = targets.to(device=device)
        
#         data = data.reshape(64, 1, 2, 2048)
        
#         scores = model(data)
#         loss = criterion(scores, targets)
        
#         optimizer.zero_grad()
#         loss.backward()
        
#         optimizer.step()

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=64
)

Epoch 1/10
703/703 [==============================] - 182s 256ms/step - loss: 6.9134 - accuracy: 0.0017
Epoch 2/10
703/703 [==============================] - 173s 247ms/step - loss: 6.9085 - accuracy: 0.0020
Epoch 3/10
703/703 [==============================] - 172s 245ms/step - loss: 6.9084 - accuracy: 0.0020
Epoch 4/10
703/703 [==============================] - 173s 245ms/step - loss: 6.9077 - accuracy: 0.0020
Epoch 5/10
703/703 [==============================] - 173s 246ms/step - loss: 6.9075 - accuracy: 0.0020
Epoch 6/10
703/703 [==============================] - 175s 249ms/step - loss: 6.9073 - accuracy: 0.0020
Epoch 7/10
703/703 [==============================] - 176s 250ms/step - loss: 6.9072 - accuracy: 0.0020
Epoch 8/10
703/703 [==============================] - 191s 271ms/step - loss: 6.9072 - accuracy: 0.0020
Epoch 9/10
703/703 [==============================] - 192s 273ms/step - loss: 6.9072 - accuracy: 0.0020
Epoch 10/10
703/703 [==============================] - 188s 267m